In [1]:
import os
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, DateTime, func, cast, text, select
from sqlalchemy.dialects import postgresql
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.orm import Session, sessionmaker
from contextlib import contextmanager
from dataclasses import dataclass

In [3]:
database_url = os.environ.get("DATABASE_URL")
engine = create_engine(
    database_url, 
    # echo=True,
)
engine

Engine(postgresql+psycopg://admin:***@db:5432/db)

In [6]:


@dataclass
class Lock:
    locked: bool
    lock_key: int


@contextmanager
def advisory_lock(session: Session, lock_value: str, commit: bool = True):
    """
    Context manager that acquires a PostgreSQL advisory lock using a hash of the lock_value
    upon entering and releases it upon exiting, using SQLAlchemy's syntax.
    
    Args:
        session (Session): SQLAlchemy session to interact with the database.
        lock_value (str): The value (string) to hash and generate the lock key.
    
    Usage:
        with advisory_lock(session, "some_unique_value"):
            # Critical section where the lock is held
    """
    try:
        # Acquire the advisory lock using hashtext and pg_advisory_xact_lock in a single call
        lock_key_query = select(func.hashtext(lock_value))
        lock_key = session.execute(lock_key_query).scalar()
        
        lock = session.execute(
            select(func.pg_try_advisory_xact_lock(lock_key))
        ).first()[0]
        # Yield control back to the caller inside the context
        yield Lock(locked=not lock, lock_key=lock_key)
    finally:
        # The lock is automatically released when the transaction ends (no explicit release needed)
        if commit:
            session.commit()

In [ ]:
from sqlalchemy.orm import sessionmaker
import time

Session = sessionmaker(autocommit=False, autoflush=False, bind=engine)

lock_value = "some_unique_string_value"  # This string will be hashed to generate the lock key

with Session() as session:
    with advisory_lock(session, lock_value, commit=True) as lock:
        if lock.locked:
            print("Skip because locked")
        else:
            # Critical section where the advisory lock is held
            # Perform operations that need the lock
            session.execute(text('select 1'))
            time.sleep(100)
            print(lock.locked, lock.lock_key)
